In [1]:
'''memory on server is full, divert to local'''
import pandas as pd
import subprocess
from Bio import SeqIO
import itertools
from operator import add
import time
import numpy as np
import matplotlib.pylab as plt

In [2]:
CODONS = map(lambda x: ''.join(x), list(itertools.product(['A','T','G','C'],['A','T','G','C'],['A','T','G','C'])))

In [3]:
STOP_CODONS = ['TAA', 'TGA', 'TAG']

In [4]:
info = pd.read_csv('/Users/richard/Desktop/Bioinfo_Research_Place/fullTableInfoGff3GffRNAESwithDensity20151109.csv')
info.index = info.shortName

In [5]:
target = '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/'

In [6]:
## convert seq to capital letters
## filter out alternative splicing
def filter1(seq): return len(seq)%3 == 0
def filter2(seq): return seq[:3] == 'ATG'
def filter3(seq): return seq[-3:] in STOP_CODONS
def filter4(seq): return 'N' not in seq

In [7]:
def process_CDS_spark(filename):
    '''
    input: filename with path
    output: sequence contained in list
    '''
    recs = sc.parallelize(list(SeqIO.parse(filename,'fasta')))
    seqs = recs.map(lambda x: str(x.seq))
    seqs = seqs.filter(lambda x: filter1(x) and filter2(x) and filter3(x) and filter4(x))
    seqs = seqs.map(lambda x: x[3:-3].upper())
    return seqs.reduce(lambda a,b: a+b)

In [8]:
def get_condon_freqs_spark(seq):
    table = {codon:0 for codon in CODONS}
    counts = sc.parallelize(xrange(0,len(seq),3)).map(lambda i: (seq[i:i+3],1)).reduceByKey(add).collect()
    for (key,count) in counts: table[key] = count
    total = 1.*sum(table.values())
    for key in table: table[key] /= total
    return table

In [9]:
codon_freqs = [] #container for gc, condon freq pairs
problem_species = []
start = time.time()
for i,shortName in enumerate(info.index):
    try:
        filename = "{}{}/nuc_seq.fna".format(target,shortName)
        seqs = process_CDS_spark(filename)
        codon_freqs.append(get_condon_freqs_spark(seqs))
        print i, shortName, 'time spent so far:', time.time()-start 
    except:
        print 'problem with {}'.format(shortName)
        problem_species.append(shortName)

0 Aaoar1 time spent so far: 4.53664016724
1 Acain1 time spent so far: 6.48990106583
2 Acema1 time spent so far: 11.9513840675
3 Aciri1_iso time spent so far: 14.38402915
4 Aciri1_meta time spent so far: 16.7811620235
5 Acral2 time spent so far: 19.0870499611
6 Acrst1 time spent so far: 22.6667020321
7 Agabi_varbisH97_2 time spent so far: 25.3175370693
8 Agabi_varbur_1 time spent so far: 28.0251300335
9 Agahy1 time spent so far: 29.6735541821
10 Altal1 time spent so far: 33.3910541534
11 Altbr1 time spent so far: 35.9695961475
12 Amamu1 time spent so far: 39.6023840904
13 Amath1 time spent so far: 42.1930861473
14 Amnli1 time spent so far: 46.642291069
15 Amore1 time spent so far: 50.505325079
16 Antav1 time spent so far: 54.7019581795
17 Antlo1 time spent so far: 55.175219059
18 Antsi1 time spent so far: 58.0907580853
19 Apimo1 time spent so far: 62.9112520218
20 Aplpr1 time spent so far: 66.0141820908
21 Armga1 time spent so far: 73.3742740154
22 Armme1_1 time spent so far: 76.5064020

In [10]:
#filter shortNames
shortNames = list(info.index)
for shortName in problem_species:
    shortNames.remove(shortName)

In [16]:
#point out problem species
pd.DataFrame(problem_species).to_csv('problem_species.csv',index=False, header = None)

In [11]:
## make table
## shortName, gc, codon vector
table_entry = []
for i,shortName in enumerate(shortNames):
    condon_vec = [codon_freqs[i][condon] for condon in CODONS]
    condon_vec.insert(0,info.ix[shortName].gc)
    condon_vec.insert(0,shortName)
    table_entry.append(condon_vec)
    
cols = CODONS[:]; cols.insert(0, 'gc'); cols.insert(0,'shortName')

condon_freq_table = pd.DataFrame(table_entry, columns=cols)

In [12]:
## save table for future use to avoid intensive calculation
condon_freq_table.to_csv('condon_freq_table.csv',index = False)

In [ ]:
##################

## to do 



##################

In [28]:
## build regression model from table
regression_models = [np.poly1d(np.polyfit(condon_freq_table.gc, condon_freq_table[codon], 3)) for codon in CODONS]

LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
# make table with regression model predicted codon frequency
vecs = []
for gc in range(25,71):
    vec = map(lambda model: model(gc), regression_models)
    vec.insert(0,gc)
    vecs.append(vec)

In [ ]:
###### test script ############

In [13]:
filename = "{}{}/nuc_seq.fna".format(target,info.index[0])
process_CDS_spark(filename)

NameError: global name 'sc' is not defined

In [66]:
count = 1
for i,shortName in enumerate(problem_speices):
    try:
        filename = "{}{}/nuc_seq.fna".format(target,shortName)
        recs = sc.parallelize(list(SeqIO.parse(filename,'fasta')))
        seqs = recs.map(lambda x: ''.join(str(x.seq).split('N')))
#         seqs = seqs.map(filter4)#.reduce(lambda a,b: a+b)
#         seqs = process_CDS_spark(filename)
#         codon_freqs.append(get_condon_freqs_spark(seqs))
        print i, shortName
    except Exception as e:
        print count,'problem with {}'.format(shortName), e
        count+=1
#         problem_species.append(shortName)

0 Botci1
1 Chagl_1
2 Copci1
3 Cryne_H99_1
4 Fusgr1
5 Fusox1
6 Maggr1
7 Magpo1
1 problem with Picst3 [Errno 2] No such file or directory: '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/Picst3/nuc_seq.fna'
2 problem with Pospl1 [Errno 2] No such file or directory: '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/Pospl1/nuc_seq.fna'
10 Pucgr1
11 Pyrtr1
12 Sclsc1
3 problem with Sporo1 [Errno 2] No such file or directory: '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/Sporo1/nuc_seq.fna'
4 problem with Trire2 [Errno 2] No such file or directory: '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/Trire2/nuc_seq.fna'
5 problem with Ustma1 [Errno 2] No such file or directory: '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/Ustma1/nuc_seq.fna'


In [63]:
problem_speices = ['Botci1',
 'Chagl_1',
 'Copci1',
 'Cryne_H99_1',
 'Fusgr1',
 'Fusox1',
 'Maggr1',
 'Magpo1',
 'Picst3',
 'Pospl1',
 'Pucgr1',
 'Pyrtr1',
 'Sclsc1',
 'Sporo1',
 'Trire2',
 'Ustma1']
for i in range(16):
    filename = "{}{}/nuc_seq.fna".format(target,problem_speices[i])
    recs = sc.parallelize(list(SeqIO.parse(filename,'fasta')))
    seqs = recs.map(lambda x: ''.join(str(x.seq).split('N')))
    seqs = seqs.map(filter4).reduce(lambda a,b: a+b)
    print seqs

16447
11124
13393
6967
13322
17708
11054
12335


IOError: [Errno 2] No such file or directory: '/Users/richard/Desktop/Bioinfo_Research_Place/CDS_cut_Dec21/Picst3/nuc_seq.fna'

In [62]:
seqs

16447

In [41]:
test = seqs.take(1)[0]

In [50]:
filter1(test)
filter2(test)
filter3(test)
# filter4(test)

False

In [36]:
regression_models = [np.poly1d(np.polyfit(list(condon_freq_table.gc), list(condon_freq_table[codon]), 3)) for codon in CODONS]

LinAlgError: SVD did not converge in Linear Least Squares

In [38]:
x= list(condon_freq_table.gc)
y = list(condon_freq_table[codon])

In [35]:
for codon in CODONS[1:]:
    print codon
    np.polyfit(condon_freq_table.gc, condon_freq_table[codon], 3)

AAT


LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
codon_freqs = [] #container for gc, condon freq pairs
problem_species = []
start = time.time()
for i,shortName in enumerate(info.index[423:]):
    try:
        filename = "{}{}/nuc_seq.fna".format(target,shortName)
        seqs = process_CDS_spark(filename)
        codon_freqs.append(get_condon_freqs_spark(seqs))
        print i, shortName, 'time spent so far:', time.time()-start 
    except:
        print 'problem with {}'.format(shortName)
        problem_species.append(shortName)

In [ ]:
filename = "{}{}/nuc_seq.fna".format(target,info.index[423])
seqs = process_CDS_spark(filename)

In [ ]:
CDSs = (sc.parallelize(CDS_shortNames[:10])
        .map(lambda name: (info.gc[name], target+name+'/'+CDS_number[name][0][:-3]))
        .map(lambda (gc, filename): (gc, list(SeqIO.parse(filename, 'fasta')))).collect())

In [ ]:
test = list(SeqIO.parse('/storage3/w/richard/meta2016/CDS_cut/Aaoar1/nuc_seq.fna','fasta'))

In [ ]:
test1 = test[0].seq

In [ ]:
str(test1)

In [ ]:
test1[-3:] == 'TAG'

In [ ]:
test1.upper()

In [ ]:
for i in range(0,len(test1),3):
    print test1[i:i+3]